# Semi-supervised learning

In [1]:
### change into configuration suitable for you
%env THEANO_FLAGS='device=gpu0','floatX=float32'

import os
import os.path as osp

import numpy as np

import theano
import theano.tensor as T
import lasagne

%matplotlib inline

import matplotlib.pyplot as plt

env: THEANO_FLAGS='device=gpu0','floatX=float32'


Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5005)


## Getting MNIST

In [2]:
%%sh

wget -q -nc https://raw.githubusercontent.com/amitgroup/amitgroup/master/amitgroup/io/mnist.py

In [3]:
### http://g.sweyla.com/blog/2012/mnist-numpy/
import mnist

In [4]:
%%sh

mkdir -p mnist && {
    cd mnist;
    wget -q -nc http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz &&
    wget -q -nc http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz &&
    wget -q -nc http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz &&
    wget -q -nc http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz &&
    gunzip *.gz
}

gzip: t10k-images-idx3-ubyte already exists;	not overwritten
gzip: t10k-labels-idx1-ubyte already exists;	not overwritten
gzip: train-images-idx3-ubyte already exists;	not overwritten
gzip: train-labels-idx1-ubyte already exists;	not overwritten


In [5]:
X, y = mnist.load_mnist(dataset='training', path='mnist/')
X = X.reshape(-1, 1, 28, 28).astype('float32')

X_test, y_test = mnist.load_mnist(dataset='testing', path='mnist/')
X_test = X_test.reshape(-1, 1, 28, 28).astype('float32')

In order to simulate semisupervised problem only a small fraction of training samples is left.

In [45]:
from sklearn.cross_validation import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, train_size=0.05)

In [46]:
X_train.shape

(3000, 1, 28, 28)

## Direct learning

In [54]:
from sklearn.linear_model import LogisticRegression

In [57]:
for C in 1.0 / np.exp(np.linspace(-2, 3, num=10)):
    lr = LogisticRegression(C=C, solver='newton-cg', multi_class='multinomial')
    lr.fit(X_train.reshape(-1, 28 * 28), y_train)

    predicted = lr.predict(X_val.reshape(-1, 28 * 28))
    print "%.1e: %.3f" % (C, np.mean(predicted == y_val))

7.4e+00: 0.878
4.2e+00: 0.880
2.4e+00: 0.882
1.4e+00: 0.884
8.0e-01: 0.886
4.6e-01: 0.889
2.6e-01: 0.891
1.5e-01: 0.892
8.7e-02: 0.893
5.0e-02: 0.892


In [58]:
from ae import CAE
cae = CAE(n_codes=25)
cae.load('CAE_25.npz')

In [59]:
for C in 1.0 / np.exp(np.linspace(-2, 3, num=10)):
    lr = LogisticRegression(C=C, solver='newton-cg', multi_class='multinomial')
    lr.fit(cae.encode(X_train), y_train)

    predicted = lr.predict(cae.encode(X_val))
    print "%.1e: %.3f" % (C, np.mean(predicted == y_val))

7.4e+00: 0.930
4.2e+00: 0.930
2.4e+00: 0.931
1.4e+00: 0.932
8.0e-01: 0.932
4.6e-01: 0.932
2.6e-01: 0.933
1.5e-01: 0.932
8.7e-02: 0.931
5.0e-02: 0.929


In [60]:
C = 8.7e-02

lr = LogisticRegression(C=C, solver='newton-cg', multi_class='multinomial')
lr.fit(X_train.reshape(-1, 28 * 28), y_train)

predicted = lr.predict(X_test.reshape(-1, 28 * 28))
print "%.1e: %.3f" % (C, np.mean(predicted == y_test))

8.7e-02: 0.898


In [61]:
lr = LogisticRegression(C=C, solver='newton-cg', multi_class='multinomial')
lr.fit(cae.encode(X_train), y_train)

predicted = lr.predict(cae.encode(X_test))
print "%.1e: %.3f" % (C, np.mean(predicted == y_test))

8.7e-02: 0.938
